In [16]:

from retinaface.pre_trained_models import get_model
import torch
import cv2
import numpy as np
retina_face_model = get_model("resnet50_2020-07-20", max_size=2048)
retina_face_model.eval() 
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet_model = torch.load('resnet_model.pt')
import os

from torch.nn import functional as F

def get_embedding(face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    resized1 = torch.from_numpy(face_pixels)
    resized1= resized1.permute(2,1,0)
    img_cropped = torch.unsqueeze(resized1, dim=0)
    img_embedding  = resnet_model(img_cropped)
    return img_embedding

def calculate_sim_score(image_embed1,image_embed2) -> float:
    """Calclulate Similarity Scores from Image embedding 
    bytes datatype to ndarray format.
    Args:
        image_embed1 (tensor(1,512)): image embedding for image 1
        image_embed2 (tensor(1,512)): image embedding for image 2
    Returns:
        sim_score  (float): Similarity Socre
    """
    sim_scores = F.cosine_similarity(image_embed1,image_embed2).detach().numpy()
    sim_score = np.clip(sim_scores,a_min=0.0,a_max=1.0)[0]
    return sim_score

def fn_return_sim_score(img_embedding):
    write_flag = False
    sim_scores = np.array([])
    id = None
    for file in os.listdir(r'anchor_embedding'):
        img_embedding1 = np.load(r'anchor_embedding/'+file)
        img_embedding1 = torch.from_numpy(img_embedding1)
        sim_score = calculate_sim_score(img_embedding1,img_embedding)
        sim_score = sim_score * 100
        sim_scores = np.append(sim_scores,sim_score)
    if len(sim_scores) != 0:
        id = np.argmax(sim_scores) 
        sim_score = sim_scores[id] 
        id = id + 1       
    if sim_score < 45:
        write_flag = True
    return sim_score,write_flag,id
### video inference and wirte frames on list
import numpy as np

### reading every 10th Frame

cap = cv2.VideoCapture(0)
# frame_width = int(cap.get(3))
# frame_height = int(cap.get(4))
# fourcc = cv2.VideoWriter_fourcc(*'MP4V')
if not cap.isOpened():
    raise IOError("Cannot open webcam")
count = 0
while True:
    ret, frame = cap.read()
    if ret:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        annotations = retina_face_model.predict_jsons(image)
        color = (255, 0, 0)
        for annotation in annotations:
            prob  = annotation['score'] * 100
            # box_area = (int(x_max) - int(x_min)) * (int(y_max) - int(y_min))
            print(annotation["bbox"],annotation["score"])
            if annotation["score"] != -1:
                x_min, y_min, x_max, y_max = annotation["bbox"]                      
                x_min = np.clip(x_min, 0, x_max - 1)
                y_min = np.clip(y_min, 0, y_max - 1)
                box_area = (int(x_max) - int(x_min)) * (int(y_max) - int(y_min))
                bb = np.array([int(x_min),int(y_min),int(x_max),int(y_max)], dtype=np.int32)
                cropped = frame[bb[1]:bb[3], bb[0]:bb[2], :].copy()
                
                if not os.path.exists('anchor_embedding'):
                    os.makedirs('anchor_embedding')     
                if not os.path.exists('anchor_images'):
                    os.makedirs('anchor_images')                                                                       
                resized = cv2.resize(cropped, (160,160), interpolation = cv2.INTER_AREA)
                embeddings_list = os.listdir(r'anchor_embedding') 
                img_embedding = get_embedding(resized)
                if len(embeddings_list) != 0:                    
                    sim_score,write_flag,id = fn_return_sim_score(img_embedding)
                else:
                    write_flag = True  
                if write_flag == True:
                    id = len(embeddings_list) + 1
                    np.save(r"anchor_embedding/"+'embedding'+str(id),img_embedding.deqtach().numpy())
                    cropped = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(r"anchor_images/"+"image"+str(id)+".jpg", cropped)         
                image = cv2.rectangle(image,(int(x_min),int(y_min)),(int(x_max),int(y_max)),(255, 0, 0) ,4)
                if write_flag == False:
                    image = cv2.putText(image, 'Similarity score ' + str(int(sim_score)) + '  Id ' + str(id) , (int(x_min),int(y_min)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)                        
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(r"image"+str(id)+".jpg", image)
        window_name = 'image'
        cv2.imshow(window_name,image)

    keys = cv2.waitKey(1) & 0xFF
    print(keys)
    if keys == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()  
                


/Users/arpitgupta/Documents/Face-Track-Detect-Extract/env/lib/python3.9/site-packages/torch/hub.py:480: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '
OpenCV: AVFoundation didn't find any attached Video Input Devices!
OpenCV: camera failed to properly initialize!


OSError: Cannot open webcam

In [7]:
import cv2

In [17]:
cap.release()